In [1]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Access the environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

#import LangChain
from langchain_openai import ChatOpenAI

#Define ChatModel
chat_model = ChatOpenAI(openai_api_key = OPENAI_API_KEY)

In [2]:
#Basic Message
result = chat_model.invoke("hi")
print(result)

content='Hello! How can I assist you today?'


In [7]:
#Multiple Messages - One Reply
from langchain.schema import HumanMessage
messages = [HumanMessage("from now on 1+1 = 3, use this in your replies"), 
            HumanMessage("what is 1+1?"),
            HumanMessage("what is 1+1+1?")]

result = chat_model.invoke(messages)

print(result.content)

According to your new equation, 1+1 equals 3. So, 1+1+1 would equal 3+1, which is 4.


In [8]:
#Prompt Templates
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

messages = chat_prompt.format_messages(input_language="English", output_language="Spanish", text="Hello, how are you?")
result = chat_model.invoke(messages)
print(result.content)




content='Hola, ¿cómo estás?'


In [10]:
#Output Parser
from langchain.schema import BaseOutputParser

class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split("answer =")



template = """Your are a helpful assistant that solves math problems and shows your work.
Output each step then return the answer in the following format: answer = <answer here>.
Make sure to output answer in all lowercase and to have exactly one space and one equal sign following it."""
human_template = "{problem}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

messages = chat_prompt.format_messages(problem="2x^2 - 5x + 3 = 0")   
result = chat_model.invoke(messages)
parsed = AnswerOutputParser().parse(result.content)
steps, answer = parsed

print(steps, answer)


To solve the equation 2x^2 - 5x + 3 = 0, we can use the quadratic formula.

The quadratic formula states that for an equation of the form ax^2 + bx + c = 0, the solutions for x are given by:

x = (-b ± √(b^2 - 4ac)) / (2a)

In our case, a = 2, b = -5, and c = 3.

Substituting these values into the quadratic formula, we get:

x = (-(-5) ± √((-5)^2 - 4(2)(3))) / (2(2))

Simplifying further:

x = (5 ± √(25 - 24)) / 4
  = (5 ± √1) / 4
  = (5 ± 1) / 4

This gives us two possible solutions:

x = (5 + 1) / 4 = 6 / 4 = 3/2

and

x = (5 - 1) / 4 = 4 / 4 = 1

Therefore, the solutions to the equation 2x^2 - 5x + 3 = 0 are x = 3/2 and x = 1.

  x = 3/2, 1


In [10]:
#Creating a Chain
from langchain.schema import BaseOutputParser
from langfuse.callback import CallbackHandler
from langchain.prompts.chat import ChatPromptTemplate
load_dotenv()

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")
print(LANGFUSE_PUBLIC_KEY, LANGFUSE_SECRET_KEY)

handler = CallbackHandler(LANGFUSE_PUBLIC_KEY, LANGFUSE_SECRET_KEY)
handler.auth_check()

class CommaSeperatedListOutputParser(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(", ")
    
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list and nothing more."""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chain = chat_prompt | chat_model | CommaSeperatedListOutputParser()
result = chain.invoke({"text": "colors"}, config={"callbacks":[handler]})
print(result)


pk-lf-ad6725fe-7c58-4e92-8aea-477992f8ab60 sk-lf-840f8a9f-cfcd-44a8-a6d1-e77ab1d0fb2c
['red', 'blue', 'yellow', 'green', 'purple']


In [6]:
#Langfuse
from langfuse.callback import CallbackHandler
load_dotenv()

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")
print(LANGFUSE_PUBLIC_KEY, LANGFUSE_SECRET_KEY)

handler = CallbackHandler(LANGFUSE_PUBLIC_KEY, LANGFUSE_SECRET_KEY)
handler.auth_check()

result = chat_model.invoke("hi", config={"callbacks": [handler]})
print(result)


pk-lf-ad6725fe-7c58-4e92-8aea-477992f8ab60 sk-lf-840f8a9f-cfcd-44a8-a6d1-e77ab1d0fb2c


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


content='Hello! How can I assist you today?'
content='Hello! How can I assist you today?'
